In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
import scipy
import csv

#import deeplabcut


Autosaving every 180 seconds


In [10]:
# Change the flag to False for Windows operation
linux = True

if linux:
    out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    video_name = os.path.join(out_dir, '2020-3-9_08_18_49_128168_compressed_snippet.avi')
    traces_means = os.path.join(out_dir, 'traces_means.npz') 
    traces_saved = os.path.join(out_dir,'traces_fixed.npz')
else:
    #out_dir = r"C:\madeline\data\'
    video_name = r"C:\madeline\data\2020-3-9_08_18_49_128168_compressed_snippet.avi"
    traces_name = r"C:\madeline\data\traces_means.npz"
    traces_saved = r"C:\madeline\data\traces_saved.npz"

    
print (video_name)
data= np.load(traces_means)

tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']
print ("DONE LOADING")

/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi
DONE LOADING


In [3]:
ids = np.arange(4)
for id_ in ids:
    for k in range(tracesx.shape[0]):
        if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
            print (k, tracesx[k,id_], tracesy[k,id_])
            tracesx[k,id_]= tracesy[k,id_] =np.nan
            print (k, tracesx[k,id_], tracesy[k,id_])
            print ('')

3135 1123.123 nan
3135 nan nan

14715 nan 92.174
14715 nan nan

779 nan 625.86
779 nan nan

13534 288.423 nan
13534 nan nan

13963 nan 244.162
13963 nan nan

9318 317.532 nan
9318 nan nan

9789 305.184 nan
9789 nan nan

14057 312.99476045543315 nan
14057 nan nan



In [4]:
# BREAKUP LARGE JUMPS
tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']

def delete_jumps(tracesx, tracesy):
    #tracesx_ip=tracesx.copy()
    #tracesy_ip=tracesy.copy()
    for k in range(tracesx.shape[1]):

        # for big jumps, set intermediate values to nans
        max_jump = 50
        for p in range(0,tracesx.shape[0]-1,1):
            distx = tracesx[p,k]-tracesx[p+1,k]
            disty = tracesy[p,k]-tracesy[p+1,k]
            dist = np.sqrt(distx**2+disty**2)
            #print ("Dist; ", dist)
            if dist>max_jump:
                #print (snippets[p,0],snippets[p,1])
                tracesx[p:p+2,k]= np.nan
                tracesy[p:p+2,k]= np.nan
                #print ("found large jump: ", p/25.)
            #else:
            #    print (dist, "not merging")


       # break
        #np.isnan(x_loc) or np.isnan(y_loc):
    print ("DONE FIXING JUMPS")
    #print ("tracex: ", tracesx[:100,0])
    
    return tracesx, tracesy

tracesx, tracesy = delete_jumps(tracesx,tracesy)
#print ("tracex_ip: ", tracesx_ip[:100,0])

#tracesx = tracesx_ip.copy()
#tracesy = tracesy_ip.copy()


DONE FIXING JUMPS


In [5]:
# INTERPOLATE TRACES
# tracesx = data['tracesx_mean']
# tracesy = data['tracesy_mean']

# TODO Add time threshold also; 
def interpolate_traces(tracesx, tracesy, dist_threshold=10., time_threshold=10):
    print (tracesx.shape)
    tracesx_ip=tracesx.copy()
    tracesy_ip=tracesy.copy()
    for k in range(tracesx_ip.shape[1]):
        start = False
        end = False
        idx = np.where(~np.isnan(tracesx_ip[:,k]))[0]

        # make snippets
        snippets=[]
        locs = []
        start = idx[0]
        #for p in range(1,idx.shape[0],1):
        p=1
        ctr=0
        while True:
            if idx[p]-idx[p-1]>1:
                end = idx[p-1]
                snippets.append([start,end])
                start = idx[p]   

                # find locations of start and end
                locs.append([])
                locs[ctr].append([tracesx_ip[start,k], tracesy_ip[start,k]])
                locs[ctr].append([tracesx_ip[end,k], tracesy_ip[end,k]])

                # bump 2 over
                p+=2
                ctr+=1
            else:
                p+=1

            if p>=idx.shape[0]:
                break

        snippets = np.array(snippets)
        #print (snippets.shape)
        #print (snippets)
        locs = np.array(locs)
    #     print (locs.shape)
    #     print (locs[0])

        # fill in space between snippets if euclidean distance < threshold
        #threshold = 10.
        # AND IF NOT TOO FAR APART; 
        for p in range(snippets.shape[0]-1):
            # Skip if interpolate across too long a sgement
            if (snippets[p,0] - snippets[p-1,1])>time_threshold:
                continue
            dist = np.linalg.norm(locs[p,1]-locs[p+1,0])
            
            #print ("Dist; ", dist, locs[p,1],locs[p+1,0])
            if dist<dist_threshold:
                #print (snippets[p,0],snippets[p,1])
                tracesx_ip[snippets[p-1,1]:snippets[p,0]+1,k]= tracesx_ip[snippets[p-1,1],k]
                tracesy_ip[snippets[p-1,1]:snippets[p,0]+1,k]= tracesy_ip[snippets[p-1,1],k]
            #else:
            #    print (dist, "not merging")

    return tracesx_ip, tracesy_ip

dist_threshold = 10.
time_threshold = 10
tracesx, tracesy = interpolate_traces(tracesx, tracesy, dist_threshold, time_threshold)

(16572, 4)


In [6]:
# AUTO SWITCH LOCATIONS BASED ON PROXIMTIY



In [11]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

global current_index, selected_animal, tracesx, tracesy

# reload data
#tracesx = tracesx_ip.copy() #data['tracesx_mean'].copy()
#tracesy = tracesy_ip.copy() #data['tracesy_mean'].copy()


# set plotting figure
fig = plt.figure()
gs = gridspec.GridSpec(4, 4)
gs.tight_layout(fig)

clrs = ['red','blue','cyan','green']
sizes = np.ones(4,'int32')*50
sizes[0] = 250
history = np.ones((4,5),'int32')+100

# save current frame of video
current_index = []
current_index.append(0)

# save selected animal radio button 
selected_animal = []
selected_animal.append(0)

# initialize start and ends of video to be analyzed
frame_rate = 25
original_vid = cv2.VideoCapture(video_name)

# initialize video
loc = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, loc)
ret, frame = original_vid.read()
n_frames = int(original_vid.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize scatter plots
ax_video = plt.subplot(gs[:, 1:4])
img = ax_video.imshow(frame)
ax_video.set_xlim(150,1200)
ax_video.set_ylim(50,800)

axes_animals=[]
for k in range(4):
    x_loc = tracesx[0,k]
    y_loc = tracesy[0,k]
    if np.isnan(x_loc) or np.isnan(y_loc):
        x_loc=100
        y_loc=100
    axes_animals.append(ax_video.scatter(x_loc, y_loc,c=clrs[k]))

# plot traces
ax_tracesx = plt.subplot(gs[:2, :1])
ax_tracesx.set_title("DLC-x")
init_length = 10*25
t = np.arange(init_length)/25.
ax1 = []
for k in range(4):
    temp, = ax_tracesx.plot(t,tracesx[:init_length,k],c=clrs[k])
    ax1.append(temp)

ax_tracesx.set_xticks([])
ax_tracesx.set_ylim(0,1280)

# plot y traces
ax_tracesy = plt.subplot(gs[2:, :1])
ax_tracesy.set_title("DLC-y")
ax2 = []
for k in range(4):
    temp, = ax_tracesy.plot(t,tracesy[:init_length,k],c=clrs[k])
    ax2.append(temp)

ax_tracesy.set_xlabel("Time (sec)")
ax_tracesy.set_ylim(0,1024)
# add 
#ax.margins(x=0)

##location of sliders
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.01, 0.01, 0.94, 0.03], facecolor=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
#sfreq = Slider(axfreq, 'Frame',0.20, 18, valinit=0, dragging=True,  fc='blue')    
#sfreq.val=val


# update the slider
def update(val):
    global tracesx, tracesy

    #amp = samp.val
    index = int(sfreq.val)
    current_index[0] = index
    print ("current indx updated:", current_index)
    
    # grab frame and update it
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = original_vid.read()
    img.set_data(frame)

    # update scatter plots
    for k in range(4):
        x_loc = tracesx[index,k]
        y_loc = tracesy[index,k]
        
        # If no location inferred, freeze and turn off labels
        if np.isnan(x_loc) or np.isnan(y_loc):
            axes_animals[k].set_alpha(0.0)
            continue
        else:
            axes_animals[k].set_alpha(1.0)
            axes_animals[k].set_sizes([sizes[k]])
            axes_animals[k].set_offsets(np.c_[x_loc,y_loc])
    
    # update trace plots
    ax_tracesx.clear()
    ax_tracesx.set_ylim(0,1280)

    ax_tracesy.clear()
    ax_tracesy.set_ylim(0,1024)
    for k in range(4):
        ax_tracesx.plot(t+current_index[0]/25., tracesx[index:index+init_length,k],
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        #ax1[k].set_linewidth(sizes[k]//50)
        #ax2[k].set_ydata(tracesy[index:index+init_length,k])    
        #ax2[k].set_linewidth(sizes[k]//50)
        ax_tracesy.plot(t+current_index[0]/25., tracesy[index:index+init_length,k], 
                        linewidth=sizes[k]//50,
                       c=clrs[k])
    fig.canvas.draw_idle()

sfreq.on_changed(update)
#samp.on_changed(update)

# resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
# button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

# def reset(event):
#     sfreq.reset()
#     #samp.reset()
# button.on_clicked(reset)

# ANIMAL SELECTOR
labels = ["female (red)", "male (blue)", "pup1 (cyan)", "pup2 (green)"]
rax1 = plt.axes([0.01, 0.8, 0.07, 0.1], facecolor=axcolor)
animal_buttons = RadioButtons(rax1, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax1.set_title('Select')
    
def animal_select(label):
    #l.set_color(label)
    fig.canvas.draw_idle()
    sizes[:] = 50
    for k in range(4):
        if labels[k] == label:
            break
    sizes[k]=250
    selected_animal[0]=k
    update(current_index[0])

    
animal_buttons.on_clicked(animal_select)

# ANIMAL SWITCHER
rax2 = plt.axes([0.01, 0.65, 0.07, 0.1], facecolor=axcolor)
animal_switch = RadioButtons(rax2, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax2.set_title('Switch')

def animal_switch_function(label):
    #l.set_color(label)
    global tracesx, tracesy
    print ('')
    print ('')
    print ("SWITCH FUNCTION")

    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k


    # search active trace snippet
    # search forward

    def find_continous_snippet(current_index, tracesx, id_):
        end=n_frames-1
        for k in range(current_index[0],n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
                start = k+1
                break
    
        return end, start
    
    print ("switching to : ", labels[switch_id])
    print ("current index inside switch:", current_index)
    
    end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
            
    print ("Switch animal id: ", switch_id, ", start/end: ", start2, end2)
    
    end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])
    
    print ("selected animal id: ", selected_animal[0], ", start/end: ", start1, end1)
        
        
    # DATA SWITCH HAS TO BE SAFE FOR ALL DOWNSTREAM 
    # SO NEED TO CHECK THE ID THAT"S BEING SWAPPED TO MAKE SURE IT DOESN"T GET DELETED;
    # switch the data
    tempx = tracesx.copy()
    tempy = tracesy.copy()
    
    print ("Target switch tracesx: ", tempx[start1:end1, switch_id].copy())
    print ("Selected animal tracesx: ", tempx[start1:end1, selected_animal[0]].copy())

    buff = 2
    print ("Switch ANIMAL TRACES preswitch; animal: ", switch_id,
          tracesx[start1-buff:end1+buff, switch_id],
          tracesy[start1-buff:end1+buff, switch_id])
        

    # replace switch id info
    tracesx[start1:end1, switch_id] = tempx[start1:end1, selected_animal[0]].copy()
    tracesy[start1:end1, switch_id] = tempy[start1:end1, selected_animal[0]].copy()
    
    print ("Switch ANIMAL TRACES postswitch; animal: ", switch_id, 
          tracesx[start1-buff:end1+buff, switch_id],
          tracesy[start1-buff:end1+buff, switch_id])
           
    print ("Selected ANIMAL TRACES preswitch; animal: ",selected_animal[0],
          tracesx[start1-buff:end1+buff, selected_animal[0]],
          tracesy[start1-buff:end1+buff, selected_animal[0]])

    # replace currental animal info
    tracesx[start1:end1, selected_animal[0]] = tempx[start1:end1, switch_id].copy()
    tracesy[start1:end1, selected_animal[0]] = tempy[start1:end1, switch_id].copy()
    
    print ("SelectedANIMAL TRACES postswitch; animal ",selected_animal[0],
          tracesx[start1-buff:end1+buff, selected_animal[0]],
          tracesy[start1-buff:end1+buff, selected_animal[0]])
  
    # update plots
    update(current_index[0])
    
animal_switch.on_clicked(animal_switch_function)


# search for the next continuous chunk of red
def next_segment(event):
    
    # 
    start_nan = n_frames-1
    for k in range(current_index[0], n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_nan = k
            break
    print ("searching... start_nan: ", start_nan)
    
    start_nan+=1
    start_val = n_frames-1
    for k in range(start_nan, n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_val=k
            break

    print ("Fast forwarding from: ", current_index[0])
    current_index[0] = start_val
    sfreq.val = current_index[0]
    fig.canvas.draw_idle()

    print ("..... to : ", current_index[0])
    update(current_index[0])
    
    # search for the next continuous chunk of red

    
def forward_function(event):
    
    # 
    current_index[0]+=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()
    
def backward_function(event):
    
    # 
    current_index[0]-=1
    sfreq.val = current_index[0]

    update(current_index[0])
    #fig.canvas.draw_idle()

        
def del_jumps_function(event):
    
    global tracesx, tracesy
    # 
    print ("DELETING JUMPS FUCNTION")
    tracesx, tracesy = delete_jumps(tracesx,tracesy)

    update(current_index[0])
    #fig.canvas.draw_idle()
    
    
def previous_segment(event):
    
    # 
    print ("Current esgemtn: ", tracesx[current_index[0]:current_index[0]+5,
                                        selected_animal[0]])
    
    # END OF PREVIOUS NAN BLOCK
    end_nan=0
    for k in range(current_index[0], 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            end_nan = k
            break
    print ("searching... start_nan: ", end_nan)

    # START OF PREVIOUS NAN BLOCK
    start_nan=0
    for k in range(end_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            start_nan=k+1
            break

    start_data =0
    for k in range(start_nan-1, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_data=k+1 # select the very next val
            break
            
    print ("Reversing from: ", current_index[0])
    current_index[0] = start_data
    sfreq.val = current_index[0]

    print ("..... to : ", current_index[0])
    update(current_index[0])
    #fig.canvas.draw_idle()

    
def save(event):
    
    np.savez(traces_saved, 
             tracesx = tracesx,
             tracesy = tracesy)
    print ("SAVING TRACES")
        
def load_function(event):
    
    global tracesx, tracesy
    
    data = np.load(traces_saved)
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    update(current_index[0])

    print ("loaded traces")

def interpolate_function(event):
    global tracesx, tracesy
    
    dist_threshold = 10.
    time_threshold = 10
    tracesx, tracesy = interpolate_traces(tracesx, tracesy, 
                                          dist_threshold,
                                         time_threshold)
    
    update(current_index[0])
    #fig.canvas.draw_idle()
    print ("INTERPOLATING TRACES, THRESHOLD: ", dist_threshold)

def export_movie(movie):
    clrs = ['blue','red','cyan','green']

    size_vid = np.array([1280,1024])
    dot_size = 16
    
    #out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    fname_out = (out_dir+'video_labeled_exported_fixed.avi')
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

    # initialize start and ends of video to be analyzed
    start = 0
    #end = start + 10*25
    end = tracesx.shape[0]
    #end = 3000
    original_vid = cv2.VideoCapture(video_name)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    for k in range(start,end,1):    
        if k %100==0:
            print (k)

        ret, frame = original_vid.read()
        if ret==False:
            break
#         if scaling:
#             frame_cropped = frame.copy()[::4,::4]
#         else:
#             frame_cropped = frame.copy()

        # Loop over animals
        for p in range(0, tracesx.shape[1], 1):
            x = tracesx[k,p]
            y = tracesy[k,p]
            # plot large bump
            if np.isnan(x) or np.isnan(y):
                continue
            x = int(x)
            y = int(y)
            
            frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()
    cv2.destroyAllWindows()

def fix_nans(event):
    global tracesx, tracesy

    ids = np.arange(4)
    for id_ in ids:
        for k in range(tracesx.shape[0]):
            if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
                print (k, tracesx[k,id_], tracesy[k,id_])
                tracesx[k,id_]= tracesy[k,id_] =np.nan
                print ('')
                
    print ("DONE FIXING nans")
    update(current_index[0])
    
def del_singles(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-2,1):
            if np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True, 
                                                 False,
                                                 True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan
    print ("DONE DELEting singles")   
    update(current_index[0])
    
    
def del_triples(event):
    global tracesx, tracesy
    
    # check to see in any sequence of 3 if the middle si the only real value
    ids = np.arange(4)
    for id_ in ids:
        for k in range(0,tracesx.shape[0]-4,1):
            flag_ = False
            if np.all(np.isnan(tracesx[k:k+5,id_])==np.array([True,
                                                              False,False,False,
                                                              True])):
                tracesx[k:k+5,id_]=np.nan
                tracesy[k:k+5,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+4,id_])==np.array([True,
                                                              False,False,
                                                              True])):
                tracesx[k:k+4,id_]=np.nan
                tracesy[k:k+4,id_]=np.nan
                flag_ = True
                
            elif np.all(np.isnan(tracesx[k:k+3,id_])==np.array([True,
                                                              False,
                                                              True])):
                tracesx[k:k+3,id_]=np.nan
                tracesy[k:k+3,id_]=np.nan 
                flag_ = True
                
            if flag_:
                print ("deleted small segs", id_, k)
    print ("DONE DELEting triples")   
    update(current_index[0])

def del_segment(event):
    global tracesx, tracesy
    
    # search active trace snippet
    # search forward

    def find_continous_snippet(current_index, tracesx, id_):
        end=n_frames-1
        for k in range(current_index[0],n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
                start = k+1
                break
    
        return end, start
 
    #end2, start2 = find_continous_snippet(current_index, tracesx, switch_id)
    end1, start1 = find_continous_snippet(current_index, tracesx, selected_animal[0])

    print ("Selected animal id: ", selected_animal, ", start/end: ", start1, end1)

    # replace switch id info
    tracesx[start1:end1, selected_animal[0]] = np.nan
    tracesy[start1:end1, selected_animal[0]] = np.nan
  
    print ("DONE DELEting triples")   
    update(current_index[0])
    
    
    
#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.01, 0.55, 0.04, 0.04])
bnext = Button(axnext, 'Next\nSegment')
bnext.on_clicked(next_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axprev = plt.axes([0.01, 0.5, 0.04, 0.04])
bprev = Button(axprev, 'Previous\nSegment')
bprev.on_clicked(previous_segment)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)




#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axforward = plt.axes([0.01, 0.45, 0.04, 0.04])
bforward = Button(axforward, 'Step\nForward')
bforward.on_clicked(forward_function)

# BACK BUTTON
axback = plt.axes([0.01, 0.4, 0.04, 0.04])
bback = Button(axback, 'Step\nBackward')
bback.on_clicked(backward_function)
#bprev = Button(axprev, 'Previous')

# DELETE JUMPS BUTTON
axdel_jumps = plt.axes([0.01, 0.3, 0.04, 0.04])
bdeljumps = Button(axdel_jumps, 'Fix\nJumps')
bdeljumps.on_clicked(del_jumps_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axinterpolate = plt.axes([0.01, 0.25, 0.04, 0.04])
binterpoloate = Button(axinterpolate, 'Interpolate\nTraces')
binterpoloate.on_clicked(interpolate_function)
#bprev = Button(axprev, 'Previous')

# INTERPOLATE
axnans = plt.axes([0.01, 0.2, 0.04, 0.04])
bnans = Button(axnans, 'Fix\nNans')
bnans.on_clicked(fix_nans)
#bprev = Button(axprev, 'Previous')



# Delete very short segments automatically
axtriples = plt.axes([0.01, 0.15, 0.04, 0.04])
btriples = Button(axtriples, 'Del\nTriples')
btriples.on_clicked(del_triples)

# Del Segment
axdel_seg = plt.axes([0.06, 0.15, 0.04, 0.04])
bdel_seg = Button(axdel_seg, 'Del\nSegment')
bdel_seg.on_clicked(del_segment)

#bprev = Button(axprev, 'Previous')

# Export movie
axmovie = plt.axes([0.01, 0.05, 0.04, 0.04])
bmovie = Button(axmovie, 'Export\nMovie')
bmovie.on_clicked(export_movie)
#bprev = Button(axprev, 'Previous')

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axsave = plt.axes([0.06, 0.1, 0.04, 0.04])
bsave = Button(axsave, 'Save\nTraces')
bsave.on_clicked(save)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload = plt.axes([0.06, 0.05, 0.04, 0.04])
bload = Button(axload, 'Load\nTraces')
bload.on_clicked(load_function)

plt.show()

current indx updated: [0]
loaded traces
current indx updated: [11469]
current indx updated: [11488]
current indx updated: [11583]
current indx updated: [11992]
current indx updated: [12325]
current indx updated: [12440]
current indx updated: [12478]
current indx updated: [12668]
current indx updated: [12687]
current indx updated: [12706]
current indx updated: [12744]
current indx updated: [12763]
current indx updated: [12972]
current indx updated: [13067]
current indx updated: [13115]
current indx updated: [13182]
current indx updated: [13220]
current indx updated: [13248]
current indx updated: [13296]
current indx updated: [13305]
current indx updated: [13315]
current indx updated: [13334]


SWITCH FUNCTION
switching to :  male (blue)
current index inside switch: [13334]
Switch animal id:  1 , start/end:  13330 13335
selected animal id:  0 , start/end:  12870 13483
Target switch tracesx:  [267.677      267.829      262.16079261 262.16079261 262.16079261
 262.16079261 262.16079261 271.

current indx updated: [13334]
current indx updated: [13334]


SWITCH FUNCTION
switching to :  female (red)
current index inside switch: [13334]
Switch animal id:  0 , start/end:  13330 13335
selected animal id:  0 , start/end:  13330 13335
Target switch tracesx:  [279.25566778 268.203      268.404      268.542      268.568     ]
Selected animal tracesx:  [279.25566778 268.203      268.404      268.542      268.568     ]
Switch ANIMAL TRACES preswitch; animal:  0 [         nan          nan 279.25566778 268.203      268.404
 268.542      268.568               nan          nan] [    nan     nan 628.548 637.528 637.562 637.562 637.562     nan     nan]
Switch ANIMAL TRACES postswitch; animal:  0 [         nan          nan 279.25566778 268.203      268.404
 268.542      268.568               nan          nan] [    nan     nan 628.548 637.528 637.562 637.562 637.562     nan     nan]
Selected ANIMAL TRACES preswitch; animal:  0 [         nan          nan 279.25566778 268.203      268.404
 268.

Current esgemtn:  [nan nan nan nan nan]
searching... start_nan:  13336
Reversing from:  13336
..... to :  12871
current indx updated: [12871]


SWITCH FUNCTION
switching to :  female (red)
current index inside switch: [12871]
Switch animal id:  0 , start/end:  12870 12880
selected animal id:  1 , start/end:  12871 13329
Target switch tracesx:  [267.829      262.16079261 262.16079261 262.16079261 262.16079261
 262.16079261 271.6315     257.688      257.42386506          nan
          nan          nan          nan          nan 267.091
 267.091      267.091      262.063      262.063      251.301
 266.779               nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan 866.867      871.567
 883.317      875.714      881.696      890.68       876.325
 863.3335     

Current esgemtn:  [267.829      262.16079261 262.16079261 262.16079261 262.16079261]
searching... start_nan:  12870
Reversing from:  12871
..... to :  12796
current indx updated: [12796]
Current esgemtn:  [252.13  252.249 267.022 267.022 267.061]
searching... start_nan:  12795
Reversing from:  12796
..... to :  12686
current indx updated: [12686]
searching... start_nan:  12700
Fast forwarding from:  12686
..... to :  12796
current indx updated: [12796]
searching... start_nan:  12869
Fast forwarding from:  12796
..... to :  12871
current indx updated: [12871]
searching... start_nan:  12880
Fast forwarding from:  12871
..... to :  12885
current indx updated: [12885]
searching... start_nan:  12892
Fast forwarding from:  12885
..... to :  12914
current indx updated: [12914]
searching... start_nan:  12933
Fast forwarding from:  12914
..... to :  12981
current indx updated: [12981]
Current esgemtn:  [243.509  243.583  243.897  244.075  243.6505]
searching... start_nan:  12980
Reversing from:



SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [13638]
Switch animal id:  3 , start/end:  13639 13638
selected animal id:  1 , start/end:  13638 13678
Target switch tracesx:  [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan]
Selected animal tracesx:  [331.045  330.7835 330.0805 331.896  329.9145 327.447  329.576  327.451
 327.451  331.295  327.756  327.756  329.6775 331.688  327.578  333.044
 332.9635 333.154  331.484  331.575  333.179  333.129  332.265  332.1085
 332.1    332.854  332.854  332.854  327.792  331.322  331.322  331.322
 331.103  331.183  331.567  332.1735 332.3515 332.4755 331.668  332.431 ]
Switch ANIMAL TRACES preswitch; animal:  3 [    nan     nan     nan     nan     nan     nan     nan     nan     nan
     nan     nan     nan     nan     nan     nan     nan     nan     nan
     nan     nan     nan     nan     nan     nan     n

...



SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [13767]
Switch animal id:  3 , start/end:  13768 13767
selected animal id:  1 , start/end:  13767 13768
Target switch tracesx:  [nan]
Selected animal tracesx:  [378.364]
Switch ANIMAL TRACES preswitch; animal:  3 [nan nan nan nan nan] [nan nan nan nan nan]
Switch ANIMAL TRACES postswitch; animal:  3 [    nan     nan 378.364     nan     nan] [     nan      nan 219.9015      nan      nan]
Selected ANIMAL TRACES preswitch; animal:  1 [    nan     nan 378.364     nan     nan] [     nan      nan 219.9015      nan      nan]
SelectedANIMAL TRACES postswitch; animal  1 [nan nan nan nan nan] [nan nan nan nan nan]
current indx updated: [13767]
searching... start_nan:  13767
Fast forwarding from:  13767
..... to :  13793
current indx updated: [13793]


SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [13793]
Switch animal id:  3 , start/end:  13794 13793
selected animal id:  1 , start/end:  13793 

searching... start_nan:  13851
Fast forwarding from:  13851
..... to :  13979
current indx updated: [13979]


SWITCH FUNCTION
switching to :  pup2 (green)
current index inside switch: [13979]
Switch animal id:  3 , start/end:  13980 13979
selected animal id:  1 , start/end:  13979 13983
Target switch tracesx:  [nan nan nan nan]
Selected animal tracesx:  [292.891      292.906      293.34       287.60484901]
Switch ANIMAL TRACES preswitch; animal:  3 [nan nan nan nan nan nan nan nan] [nan nan nan nan nan nan nan nan]
Switch ANIMAL TRACES postswitch; animal:  3 [         nan          nan 292.891      292.906      293.34
 287.60484901          nan          nan] [     nan      nan 252.984  252.984  252.6845 252.196       nan      nan]
Selected ANIMAL TRACES preswitch; animal:  1 [         nan          nan 292.891      292.906      293.34
 287.60484901          nan          nan] [     nan      nan 252.984  252.984  252.6845 252.196       nan      nan]
SelectedANIMAL TRACES postswitch; animal

Current esgemtn:  [354.455  361.4185 364.1945 364.522  364.5335]
searching... start_nan:  14778
Reversing from:  14779
..... to :  14744
current indx updated: [14744]
searching... start_nan:  14768
Fast forwarding from:  14744
..... to :  14779
current indx updated: [14779]
Selected animal id:  [1] , start/end:  14779 14784
DONE DELEting triples
current indx updated: [14779]
searching... start_nan:  14779
Fast forwarding from:  14779
..... to :  14789
current indx updated: [14789]
Selected animal id:  [1] , start/end:  14789 14805
DONE DELEting triples
current indx updated: [14789]
searching... start_nan:  14789
Fast forwarding from:  14789
..... to :  14807
current indx updated: [14807]
searching... start_nan:  14812
Fast forwarding from:  14807
..... to :  14814
current indx updated: [14814]
Selected animal id:  [1] , start/end:  14814 14853
DONE DELEting triples
current indx updated: [14814]
searching... start_nan:  14814
Fast forwarding from:  14814
..... to :  14859
current indx u

searching... start_nan:  15323
Fast forwarding from:  15323
..... to :  15325
current indx updated: [15325]
searching... start_nan:  15349
Fast forwarding from:  15325
..... to :  15380
current indx updated: [15380]
searching... start_nan:  15399
Fast forwarding from:  15380
..... to :  15405
current indx updated: [15405]
searching... start_nan:  15421
Fast forwarding from:  15405
..... to :  15423
current indx updated: [15423]
searching... start_nan:  15436
Fast forwarding from:  15423
..... to :  15493
current indx updated: [15493]
searching... start_nan:  15560
Fast forwarding from:  15493
..... to :  15562
current indx updated: [15562]
searching... start_nan:  15570
Fast forwarding from:  15562
..... to :  15572
current indx updated: [15572]
searching... start_nan:  15589
Fast forwarding from:  15572
..... to :  15667
current indx updated: [15667]
searching... start_nan:  15686
Fast forwarding from:  15667
..... to :  15779
current indx updated: [15779]


SWITCH FUNCTION
switching 

searching... start_nan:  16135
Fast forwarding from:  16130
..... to :  16143
current indx updated: [16143]
searching... start_nan:  16170
Fast forwarding from:  16143
..... to :  16172
current indx updated: [16172]
searching... start_nan:  16220
Fast forwarding from:  16172
..... to :  16231
current indx updated: [16231]
searching... start_nan:  16260
Fast forwarding from:  16231
..... to :  16264
current indx updated: [16264]
searching... start_nan:  16286
Fast forwarding from:  16264
..... to :  16288
current indx updated: [16288]


In [14]:
print (tracesx[2311-1:2311+2,2])

[    nan 300.674     nan]


In [32]:
id_=2
for k in range(tracesx.shape[0]):
    if np.isnan(tracesx[k,id_])!=np.isnan(tracesy[k,id_]):
        print (k, tracesx[k,id_], tracesy[k,id_])

9318 317.532 nan
9319 317.532 nan
9320 317.532 nan
9321 317.532 nan
9322 317.532 nan
9323 317.532 nan
9324 317.532 nan
9325 317.532 nan
9326 317.532 nan
9327 317.532 nan
9328 317.532 nan
9329 317.532 nan
9789 305.184 nan


In [60]:
k = 100
id_ = 0
print (np.isnan(tracesx[k:k+3,id_]))
print (np.array([False, 
                                                 True,
                                                 False]))
print (np.all(np.isnan(tracesx[k:k+3,id_])==np.array([False, 
                                                 True,
                                                 False])))

[False False False]
[False  True False]
False
